# Chr17-22, GTEx Adipose (subcutaneous) fusion weights

In [1]:
g2dir = "cd ~/causalTWAS/simulations/simulation_ashtest_20200721"

* simulate different PVEs and run mr.ash2s.

In [2]:
# need 16G
simu = "Rscript /home/simingz/causalTWAS/causal-TWAS/code/run_simulate_data.R  /home/simingz/causalTWAS/ukbiobank/ukb_chr17:22_s20000 /home/simingz/causalTWAS/fusion_weights/Adipose_Subcutaneous param-{tag}.R simu_{tag}"

* test mr.ash2s

In [3]:
# need an exclusive core for FBM version. <1 hour for one mode.
# source activate twas
mrash2 = "Rscript /home/simingz/causalTWAS/causal-TWAS/code/run_test_mr.ash2s.R  /home/simingz/causalTWAS/ukbiobank/ukb_chr17:22_s20000  ~/causalTWAS/simulations/simulation_ashtest_20200721/simu_{tag}-cis-expr.Rd ~/causalTWAS/simulations/simulation_ashtest_20200721/simu_{tag}-pheno.Rd {tag}"

In [15]:
## GWAS for all (require 48G for snp GWAS, using 5 cores.)
# source activate twas
exprgwas = "Rscript /home/simingz/causalTWAS/causal-TWAS/code/run_gwas_expr.R simu_{tag}-cis-expr.Rd simu_{tag}-pheno.Rd {tag}"
snpgwas = "Rscript /home/simingz/causalTWAS/causal-TWAS/code/run_gwas_snp.R /home/simingz/causalTWAS/ukbiobank/ukb_chr17:22_s20000 simu_{tag}-pheno.Rd {tag}"

In [5]:
# ld
ld = "Rscript /home/simingz/causalTWAS/causal-TWAS/code/run_ld.R /home/simingz/causalTWAS/ukbiobank/ukb_chr22_s20000 simu_{tag}-pheno.Rd {tag}"

* prepare param 

In [16]:
cd ~/causalTWAS/simulations/simulation_ashtest_20200721

/project2/mstephens/causalTWAS/simulations/simulation_ashtest_20200721


In [7]:
template = "param-20200721-1.R"
a = open(template).readlines()
for i in range(1,11):
    with open("param-20200721-1-" + str(i) + ".R", 'w' ) as newpar:
        a[5] = 'SED <- {sed}\n'.format(sed = str(950 + i))
        newpar.writelines(a)

* Generate command

In [25]:
tags = ['20200721-1-' + str(i) for i in [2] + list(range(4,11))]

In [26]:
tags

['20200721-1-2',
 '20200721-1-4',
 '20200721-1-5',
 '20200721-1-6',
 '20200721-1-7',
 '20200721-1-8',
 '20200721-1-9',
 '20200721-1-10']

In [27]:
with open('master_run1-x.sh', 'w') as outf:
    for tag in tags:
        outf.write(';'.join(['source activate twas', 
                   g2dir,
                   #simu.format(tag=tag),
                   #mrash2.format(tag=tag),
                   exprgwas.format(tag=tag),
                   snpgwas.format(tag=tag),
                   #ld.format(tag=tag)
                            ]) + '\n')

In [28]:
!head -n 1  master_run1-x.sh

source activate twas;cd ~/causalTWAS/simulations/simulation_ashtest_20200721;Rscript /home/simingz/causalTWAS/causal-TWAS/code/run_gwas_expr.R simu_20200721-1-2-cis-expr.Rd simu_20200721-1-2-pheno.Rd 20200721-1-2;Rscript /home/simingz/causalTWAS/causal-TWAS/code/run_gwas_snp.R /home/simingz/causalTWAS/ukbiobank/ukb_chr17:22_s20000 simu_20200721-1-2-pheno.Rd 20200721-1-2


## test SUSIE for fine mapping

In [29]:
## multiple runs
g2susiedir = "cd ~/causalTWAS/simulations/simulation_susietest_20200721"
mrashres = "~/causalTWAS/simulations/simulation_ashtest_20200721/{tag}-mr.ash2s.{tag2}"
susie = "Rscript /home/simingz/causalTWAS/causal-TWAS/code/run_test_susie.R /home/simingz/causalTWAS/ukbiobank/ukb_chr17:22_s20000  ~/causalTWAS/simulations/simulation_ashtest_20200721/simu_{tag}-cis-expr.Rd ~/causalTWAS/simulations/simulation_ashtest_20200721/simu_{tag}-pheno.Rd {prior} {region} {tag}.{tag2} 3"

In [30]:
cd ~/causalTWAS/simulations/simulation_susietest_20200721

/project2/mstephens/causalTWAS/simulations/simulation_susietest_20200721


In [31]:
tags = ['20200721-1-' + str(i) for i in [2] + list(range(4,11))]
tag2s = ['zeroes-es', 'lassoes-se', 'lassoes-es', 'zerose-es']

In [32]:
# 28G, 20min for one run
with open('susie_run1-x.sh', 'w') as outf:
    for tag in tags:
        for tag2 in tag2s:
            param = mrashres.format(tag = tag, tag2 = tag2) + ".param.txt"
            region = mrashres.format(tag = tag, tag2 = tag2)
            outf.write(';'.join(['source activate twas', 
                             g2susiedir,
                             susie.format(tag = tag, tag2 = tag2, prior = param, region = region),
                             ]) + '\n')